In [1]:
from datetime import date, datetime
import math
import ROOT
import numpy as np, pandas as pd
import bokeh
import yaml
from bokeh.io import output_notebook, show
from bokeh.models import CustomJS, MultiSelect, Button, ColumnDataSource, DateRangeSlider, CheckboxButtonGroup
from bokeh.events import ButtonClick, MenuItemClick
from bokeh.layouts import row, column
output_notebook()
ROOT.gROOT.LoadMacro("runCCDBItemList.C")
#ROOT.gROOT.LoadMacro("runCCDBDownloadTarget.C")

Welcome to JupyROOT 6.24/06


Loading BokehJS ...

0

In [2]:
ROOT.runCCDBItemList()
with open('directoryTree.txt') as directoryTree: 
    lines = directoryTree.readlines()

[INFO] Is alien token present?: 0


In [3]:
files = list()
paths = list()
time_stamps = list()
dates = list()
types = list()

active_file = ''
file_type = ''
for line in lines:
    if line[0:7] == 'qc/TPC/':
        active_file = line[10:-2]
        files.append(active_file)
    elif line[0:5] == 'Type:':
        file_type = line[6:-1]
    elif line[0] == '-':
        paths.append(active_file)
        types.append(file_type)
        time_stamp = int(line[3:-2])
        time_stamps.append(time_stamp)
        date = datetime.utcfromtimestamp(math.ceil(time_stamp)/100).strftime('%Y-%m-%d-%H:%M:%S')
        dates.append(date)
    
files_data = pd.DataFrame({'Path': paths, 'Type': types, 'TimeStamp': time_stamps, 'Date':dates})
type_set = sorted(list(set(types)))

In [4]:
type_set

['TEfficiency', 'TH1F', 'TH2F']

In [5]:
files_data['TimeStamp'].min()

160562680033

In [45]:
selected_data = []
selected_files = files_data.copy(deep=True)
selected_types = type_set
time_range = [files_data['TimeStamp'].min()*10,files_data['TimeStamp'].max()*10]
shown_files = files

def file_browser(doc):
    def update_button_label():
        label = "Download {} files".format(str(len(selected_files.index)))
        button.label = label
        
    def update_file_select_list(shown_files):
        new_list = list(set(shown_files['Path']))
        file_select.options = new_list
        file_select.value = []
        update_time_range(shown_files)
        
    def update_time_range(shown_files):
        if not shown_files.empty:
            date_range.value=[shown_files['TimeStamp'].min()*10,shown_files['TimeStamp'].max()*10]
            date_range.start=shown_files['TimeStamp'].min()*10
            date_range.end=shown_files['TimeStamp'].max()*10
    
    def download_selected(selection):
        paths2download = selection['Path'].to_list()
        ROOT.runCCDBDownloadTarget(str(paths2download)) 
        
    def select_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types
        selected_data = file_select.value
        time_condition = ((files_data['TimeStamp'] >= time_range[0]/10) & (files_data['TimeStamp'] <= time_range[1]/10))
        path_condition = files_data['Path'].map(lambda x: x in selected_data)
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        selected_files = files_data[path_condition & time_condition & type_condition]
        update_button_label()
        
    def date_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types
        time_range = date_range.value
        time_condition = ((files_data['TimeStamp'] >= time_range[0]/10) & (files_data['TimeStamp'] <= time_range[1]/10))
        path_condition = files_data['Path'].map(lambda x: x in selected_data)
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        selected_files = files_data[path_condition & time_condition & type_condition]
        update_button_label()
        
    def type_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files
        selected_types = [type_set[i] for i in checkbox.active]
        time_condition = ((files_data['TimeStamp'] >= time_range[0]/10) & (files_data['TimeStamp'] <= time_range[1]/10))
        path_condition = files_data['Path'].map(lambda x: x in selected_data)
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        selected_files = files_data[path_condition & time_condition & type_condition]
        shown_files = files_data[type_condition]
        update_file_select_list(shown_files)
        update_button_label()
        
    def button_callback(event):
        global selected_data, selected_files, files_data, time_range, selected_types
        print("Will download\n", selected_files)
        #download_selected(selected_files)   

    checkbox = CheckboxButtonGroup(labels=type_set,active=list(range(0,len(type_set))))
    checkbox.on_change("active",type_callback)
        
    date_range = DateRangeSlider(value=[files_data['TimeStamp'].min()*10,files_data['TimeStamp'].max()*10],
                                    start=files_data['TimeStamp'].min()*10, end=files_data['TimeStamp'].max()*10)
    date_range.on_change("value_throttled",date_callback)    
    
    file_select = MultiSelect(title="Files", value=[], options=files, height=200)
    file_select.on_change("value", select_callback)

    button = Button(label="Download all", button_type="success") 
    button.on_event(ButtonClick, button_callback)
    
    doc.add_root(column(file_select,checkbox,date_range,button))
show(file_browser)

Will download
               Path  Type     TimeStamp                 Date
1236  Tracks/hPtNe  TH1F  163343131937  2021-10-05-10:55:19
1237  Tracks/hPtNe  TH1F  163717135344  2021-11-17-17:49:13
1238  Tracks/hPtNe  TH1F  163784791711  2021-11-25-13:45:17
1239  Tracks/hPtNe  TH1F  163862796188  2021-12-04-14:26:01
1240  Tracks/hPtNe  TH1F  163915175088  2021-12-10-15:55:50
1241  Tracks/hPtNe  TH1F  163966613624  2021-12-16-14:48:56
1242  Tracks/hPtNe  TH1F  164001790055  2021-12-20-16:31:40
1243  Tracks/hPtNe  TH1F  163551147604  2021-10-29-12:44:36
1244  Tracks/hPtNe  TH1F  163533923080  2021-10-27-12:53:50
1245  Tracks/hPtNe  TH1F  163343989812  2021-10-05-13:18:18
1246  Tracks/hPtNe  TH1F  163343626620  2021-10-05-12:17:46
1247  Tracks/hPtNe  TH1F  162920321985  2021-08-17-12:26:59
1248  Tracks/hPtNe  TH1F  162886382710  2021-08-13-14:10:27
1249  Tracks/hPtNe  TH1F  162791966473  2021-08-02-15:54:24
1250  Tracks/hPtNe  TH1F  162884783156  2021-08-13-09:43:51
1251  Tracks/hPtNe  TH1F 

In [36]:
shown_files.empty

True

In [8]:
ROOT.runCCDBDownloadTarget(str(selected_files['Path'].to_list()))

[INFO] Is alien token present?: 0
['qc/TPC/MO/BhawaniTutorial/h2DNClustersEt', 'qc/TPC/MO/BhawaniTutorial/h2DNClustersPhiAsid', 'qc/TPC/MO/BhawaniTutorial/h2DNClustersPhiCsid']


# Workaround with hash numbers
could get hash numbers by scrapting the web address

In [3]:
!wget http://ccdb-test.cern.ch:8080/download/91218f16-6414-11ec-8339-8af603f7250c -O ./test_download/out.root
!ls -l test_download
!cat test_download/out.root

<cppyy.gbl.std.vector<string> object at 0xb6caff0>